In [3]:
import pandas as pd

First import pandas to help us work with the collision data excel file.

In [11]:
pd.set_option('display.max_columns', None)
collisions = pd.read_csv("/home/alec/Desktop/code/personal_projects/safe-path-finder/data/collisions.csv")
collisions.head(10)

/tmp/ipykernel_185056/1122874997.py:2: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  collisions = pd.read_csv("/home/alec/Desktop/code/personal_projects/safe-path-finder/data/collisions.csv")


,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,EXCEPTRSNCODE,EXCEPTRSNDESC,SEVERITYCODE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INJURIES,SERIOUSINJURIES,FATALITIES,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR,SOURCE,SOURCEDESC,ADDBY,ADDDTTM,MODBY,MODDTTM
0,-122.311548,47.607109,1,18100,18100,1799155,Unmatched,Block,NaN,16TH AVE BETWEEN E JEFFERSON ST AND E CHERRY ST,NaN,NaN,0,Unknown,NaN,0,0,0,0,0,0,0,2004/10/07 00:00:00+00,10/7/2004,Mid-Block (not related to intersection),11.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,NaN,NaN,NaN,NaN,NaN,4281024.0,NaN,NaN,NaN,0,0,N,PTCR,Police Traffic Collision Report,ECKLEYS,2004/10/21 00:00:00+00,ECKLEYS,2004/10/21 00:00:00+00
1,-122.344604,47.697445,2,19900,19900,1079890,Unmatched,Block,NaN,AURORA AVE N BETWEEN N 94TH ST AND N 95TH ST,NaN,NaN,0,Unknown,NaN,0,0,0,0,0,0,0,2004/09/23 00:00:00+00,9/23/2004,Mid-Block (but intersection related),16.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE ...",NaN,NaN,NaN,NaN,NaN,NaN,4267039.0,NaN,NaN,NaN,0,0,N,PTCR,Police Traffic Collision Report,ECKLEYS,2004/10/13 00:00:00+00,ECKLEYS,2004/10/13 00:00:00+00
2,-122.288424,47.564962,3,101200,101200,3296458,Matched,Block,NaN,RAINIER AVE S BETWEEN S ADAMS ST AND S GENESEE...,NaN,NaN,2,Injury Collision,Rear Ended,2,0,0,2,2,0,0,2009/06/14 00:00:00+00,6/14/2009 5:20:00 AM,Mid-Block (but intersection related),14.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",NaN,0,Overcast,Dry,Dawn,NaN,9165016.0,Y,14.0,From same direction - both going straight - on...,0,0,N,PTCR,Police Traffic Collision Report,GIVENSA,2009/06/16 00:00:00+00,STATE INTERFACE,2009/11/12 00:00:00+00
3,NaN,NaN,4,53800,53800,2609018,Matched,Block,NaN,BROAD ST BETWEEN HARRISON ST AND BROAD ST EB O...,NaN,NaN,2,Injury Collision,Left Turn,2,0,0,2,2,0,0,2006/10/04 00:00:00+00,10/4/2006,Mid-Block (but intersection related),16.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE ...",NaN,0,Clear,Dry,Daylight,NaN,6277029.0,NaN,15.0,From same direction - one left turn - one stra...,0,0,N,PTCR,Police Traffic Collision Report,ECKLEYS,2006/10/30 00:00:00+00,STATE INTERFACE,2007/05/15 00:00:00+00
4,-122.289571,47.600070,5,65000,65000,2620883,Matched,Block,NaN,S FRINK PL BETWEEN S WASHINGTON ST AND LAKE WA...,NaN,NaN,1,Property Damage Only Collision,Other,1,0,0,1,0,0,0,2007/03/24 00:00:00+00,3/24/2007 5:34:00 PM,Mid-Block (not related to intersection),26.0,MOTOR VEHICLE STRUCK OBJECT IN ROAD,NaN,0,Raining,Wet,Daylight,NaN,7083003.0,NaN,50.0,Fixed object,0,0,N,PTCR,Police Traffic Collision Report,LUKEL,2007/03/28 00:00:00+00,STATE INTERFACE,2007/05/16 00:00:00+00
5,-122.355852,47.681804,6,1043,1043,3615237,Unmatched,Block,NaN,N 73RD ST BETWEEN PALATINE AVE N AND GREENWOOD...,NaN,NaN,0,Unknown,NaN,0,0,0,0,0,0,0,2013/03/24 00:00:00+00,3/24/2013,Mid-Block (not related to intersection),13.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,N,CVCR,Citizen Vehicle Collision Report,WANWIGN,2013/04/01 00:00:00+00,ZUNIGAR1,2014/05/27 00:00:00+00
6,-122.375447,47.697887,7,47500,47500,2026892,Matched,Intersection,37223.0,MARY AVE NW AND NW 95TH ST,NaN,NaN,1,Property Damage Only Collision,Other,1,0,0,1,0,0,0,2005/12/14 00:00:00+00,12/14/2005 3:15:00 PM,At Intersection (intersection related),28.0,MOTOR VEHICLE RAN OFF ROAD - HIT FIXED OBJECT,NaN,0,Clear,Dry,Daylight,NaN,5348005.0,NaN,50.0,Fixed object,0,0,N,PTCR,Police Traffic Collision Report,AGATSUC,2006/06/02 00:00:00+00,STATE INTERFACE,2007/05/10 00:00:00+00
7,-122.344651,47.701430,8,50600,50600,C662927,Matched,Intersection,37301.0,AURORA AVE N AND N 100TH ST,NaN,NaN,1,Property Damage Only Collision,Right Turn,2,0,0,2,0,0,0,2006/01/20 00:00:00+00,1/20/2006 10:40:00 PM,At Intersection (intersection related),11.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ..."

Here we read our collisions data file into the program as a DataFrame. We set the pandas options to display the max columns so pandas doesn't hide any of the columns when we display the DataFrame. Some important columns to look at for the machine learning model are location, time, weather, road conditions, and light conditions.

Categorical Variables: Weather, Road Conditions, light conditions, time